# 사용자가 업로드한 사진에 대해 Instance Segmentaiton 진행

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/2021_capstone/cs_Local_Style_Transfer

/content/drive/MyDrive/2021_capstone/cs_Local_Style_Transfer


In [3]:
ls

backbone.py              local_stylized.jpg              run_coco_eval.py
data/                    logs/                           style_transfer.py
eval.py                  mask_data.npy                   train.py
fg_bg/                   merge_fg_bg.py                  utils/
labelme2coco/            pth/                            weights/
layers/                  __pycache__/                    yolact.py
LICENSE                  python_execution_command.ipynb
local_style_transfer.py  README.md


## 1. Custom Data Training (Make pth file)

In [4]:
# !python train.py --config=yolact_resnet50_cig_indoor_config --batch_size=2
# !python train.py --config=yolact_resnet50_cig_indoor_config --resume=weights/resnet50-19c8e357.pth --start_iter=0 --batch_size=3

## 2. Custom Train된 pth file로 Test

In [5]:
!python eval.py --trained_model=weights/yolact_plus_resnet50_cig_indoor_78_869_interrupt.pth --config=yolact_resnet50_cig_indoor_config --score_threshold=0.3 --top_k=15 --image=data/train/images/kor_bedroom1.jpg:data/eval_result/kor_bedroom1.jpg --display_bbox=False --display_text=False 

torch.cuda.is_available :  True
Loading model...path :  weights/yolact_plus_resnet50_cig_indoor_78_869_interrupt.pth
 Done.
bbox_label_list :  [[array([  0, 241, 307, 482]), 'pillow00'], [array([319, 398, 386, 486]), 'tv01'], [array([269, 356, 324, 458]), 'tv02'], [array([301, 343, 403, 498]), 'tv03']]


# 사용자가 원하는 object 및 스타일을 선택한 이후 Local Style Transfer

In [6]:
targets = ['tv01', 'tv02', 'tv03', 'pillow00'] # multiple objects selected
# targets = ['pillow00'] # one object selected

### 1. 사용자가 선택한 object의 fg, bg 이미지 전처리

In [7]:
!python merge_fg_bg.py --image_path './fg_bg'

Traceback (most recent call last):
  File "merge_fg_bg.py", line 53, in <module>
    if len(targets)>=2:
NameError: name 'targets' is not defined


### 2. 사용자가 입력한 테마 이미지 혹은 설정한 스타일에 따른 전체 스타일 변화 이미지 생성

In [17]:
!python style_transfer.py --vgg_path './pth/style_transfer/vgg_normalised.pth' --decoder_path './pth/style_transfer/decoder.pth' --content_image './data/train/images/kor_bedroom1.jpg' --style_image './data/style/classic_brown.jpg' --output_path './fg_bg/stylized.jpg'

### 3. 전처리된 이미지 조합하여 Local Style Transferred 이미지 생성

In [18]:
!python local_style_transfer.py --fg_image_path "./fg_bg/fg_result.jpg" --bg_image_path "./fg_bg/bg_result.jpg" --stylized_image_path "./fg_bg/stylized.jpg" --output_path "./local_stylized.jpg"